In [1]:
import string
from datetime import datetime
import os
import numpy as np
import pandas as pd
import json
from collections import defaultdict
from Queue import Queue

from scipy.sparse.linalg import svds

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.models.word2vec import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import plotly
plotly.tools.set_credentials_file(username='amcire96', api_key='sej35ud4YbSOfIshhhZg')
# print(plotly.__version__)

import plotly.plotly as py
from plotly.graph_objs import *
import requests
requests.packages.urllib3.disable_warnings()

In [3]:
reviews = []
with open("yelp_academic_dataset_review.json", "r") as f:
    for line in f:
        json_line = json.loads(line)
        reviews.append(json_line)

In [4]:
print(len(reviews))
reviews[0]

4153150


{u'business_id': u'2aFiy99vNLklCx3T_tGS9A',
 u'cool': 0,
 u'date': u'2011-10-10',
 u'funny': 0,
 u'review_id': u'NxL8SIC5yqOdnlXCg18IBg',
 u'stars': 5,
 u'text': u"If you enjoy service by someone who is as competent as he is personable, I would recommend Corey Kaplan highly. The time he has spent here has been very productive and working with him educational and enjoyable. I hope not to need him again (though this is highly unlikely) but knowing he is there if I do is very nice. By the way, I'm not from El Centro, CA. but Scottsdale, AZ.",
 u'type': u'review',
 u'useful': 0,
 u'user_id': u'KpkOkG6RIf4Ra25Lhhxf1A'}

In [5]:
businesses = []
with open("yelp_academic_dataset_business.json", "r") as f:
    for line in f:
        json_line = json.loads(line)
        businesses.append(json_line)

In [6]:
print(len(businesses))
businesses[0]

144072


{u'address': u'227 E Baseline Rd, Ste J2',
 u'attributes': [u'BikeParking: True',
  u'BusinessAcceptsBitcoin: False',
  u'BusinessAcceptsCreditCards: True',
  u"BusinessParking: {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
  u'DogsAllowed: False',
  u'RestaurantsPriceRange2: 2',
  u'WheelchairAccessible: True'],
 u'business_id': u'0DI8Dt2PJp07XkVvIElIcQ',
 u'categories': [u'Tobacco Shops', u'Nightlife', u'Vape Shops', u'Shopping'],
 u'city': u'Tempe',
 u'hours': [u'Monday 11:0-21:0',
  u'Tuesday 11:0-21:0',
  u'Wednesday 11:0-21:0',
  u'Thursday 11:0-21:0',
  u'Friday 11:0-22:0',
  u'Saturday 10:0-22:0',
  u'Sunday 11:0-18:0'],
 u'is_open': 0,
 u'latitude': 33.3782141,
 u'longitude': -111.936102,
 u'name': u'Innovative Vapors',
 u'neighborhood': u'',
 u'postal_code': u'85283',
 u'review_count': 17,
 u'stars': 4.5,
 u'state': u'AZ',
 u'type': u'business'}

In [7]:
valid_business_ids = set()
business_id_to_index = {}
business_index_to_bid = {}
c = 0
for business in businesses:
    if business["review_count"] >= 1000:
        valid_business_ids.add(business["business_id"])
        if business["business_id"] not in business_id_to_index:
            business_id_to_index[business["business_id"]] = c
            business_index_to_bid[c] = business["business_id"]
            c += 1
print(len(valid_business_ids))
valid_business_ids

177


{u'--9e1ONYQuAa-CB_Rrw7Tw',
 u'-ed0Yc9on37RoIoG2ZgxBA',
 u'0FUtlsQrJI7LhqDPxLumEw',
 u'0NmTwqYEQiKErDv4a55obg',
 u'0W4lkclzZThpx3V65bVgig',
 u'2iTsRqUsPGRH1li1WVRvKQ',
 u'2weQS-RnoOBhb1KsHKyoSQ',
 u'3BCsAgo_1i4xMuTyLKMLRQ',
 u'3GEEy7RP6e4bT4LAiWFMFQ',
 u'3N9U549Zse8UP-MwKZAjAQ',
 u'3kdSl5mo9dWC4clrQjEDGg',
 u'3l54GTr8-E3XPbIxnF_sAA',
 u'3xmfT7l3xNH5LK1dLzfvGw',
 u'4GXII-GU7S0ZyU6ElkhscQ',
 u'4JNXUYY8wbaaDmk3BPzlWw',
 u'4k3RlMAMd46DZ_JyZU0lMg',
 u'5LNZ67Yw9RD6nf4_UhXOjw',
 u'5T6kFKFycym_GkhgOiysIw',
 u'5iSmZO0SrKU6EoXK_1M8Kw',
 u'5shgJB7a-2_gdnzc0gsOtg',
 u'6Ct57qgmXwOnzfSZoUGh0Q',
 u'6Q7-wkCPc1KF75jZLOTcMw',
 u'7fxebHYUwIF6CakxSr70iQ',
 u'7sPNbCx7vGAaH7SbNPZ6oA',
 u'7sb2FYLS2sejZKxRYF9mtg',
 u'9SU7ZZhaFUJJ6m2k5HKHeg',
 u'9a3DrZvpYxVs3k_qwlCNSw',
 u'A-uZAD4zP3rRxb44WUGV5w',
 u'A5Rkh7UymKm0_Rxm9K2PJw',
 u'AV6weBrZFFBfRGCbcRGO4g',
 u'Az_60nNuh1FH8Ds8oasZjw',
 u'BH9z7IJ4zydAqgwsbqoVZQ',
 u'BIBWGO_r_1znnlmLbp4Nxg',
 u'BLIJ-p5wYuAhw6Pp6mh6mw',
 u'ByFMv3p5X1aNeZhU61rDcA',
 u'CiYLq33nAyghFkUR1

In [8]:
only_text = []
ordered_review_ids = []
ordered_business_id_indices = []
bid_to_rid_dict = defaultdict(list)
rid_to_bid_dict = {}
for review in reviews:
    if review["business_id"] in valid_business_ids:
        only_text.append(review["text"])
        ordered_review_ids.append(review["review_id"])
        ordered_business_id_indices.append(business_id_to_index[review["business_id"]])
        bid_to_rid_dict[review["business_id"]].append(review["review_id"])
        rid_to_bid_dict[review["review_id"]] = review["business_id"]
print(len(only_text))
print(len(ordered_review_ids))

315002
315002


In [9]:
tfidf_vectorizer = TfidfVectorizer(input=only_text, lowercase=True, stop_words="english", max_df=0.9, min_df=100)
tfidf_review_text = tfidf_vectorizer.fit_transform(only_text)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

print(tfidf_review_text.shape)

(315002, 9071)


In [10]:
count_vectorizer = CountVectorizer(input=only_text, lowercase=True, stop_words="english", max_df=0.9, min_df=100)
count_review_text = count_vectorizer.fit_transform(only_text)
count_feature_names = count_vectorizer.get_feature_names()

print(count_review_text.shape)

(315002, 9071)


In [11]:
print(count_feature_names)

[u'00', u'000', u'00am', u'00pm', u'05', u'07', u'08', u'09', u'10', u'100', u'1000', u'101', u'1030', u'105', u'108', u'109', u'10am', u'10min', u'10oz', u'10pm', u'10th', u'10x', u'11', u'110', u'115', u'11am', u'11pm', u'11th', u'12', u'120', u'1200', u'125', u'12am', u'12oz', u'12pm', u'12th', u'13', u'130', u'14', u'140', u'14th', u'15', u'150', u'1500', u'15min', u'15mins', u'15pm', u'15th', u'16', u'160', u'16oz', u'16th', u'17', u'170', u'1762', u'17th', u'18', u'180', u'18th', u'19', u'19th', u'1am', u'1hr', u'1pm', u'1st', u'20', u'200', u'2000', u'2005', u'2006', u'2007', u'2008', u'2009', u'2010', u'2011', u'2012', u'2013', u'2014', u'2015', u'2016', u'20min', u'20mins', u'20s', u'20th', u'21', u'21st', u'22', u'22nd', u'23', u'230', u'23rd', u'24', u'24hr', u'24hrs', u'24th', u'25', u'250', u'25th', u'26', u'26th', u'27', u'27th', u'28', u'28th', u'29', u'29th', u'2am', u'2nd', u'2pm', u'2x', u'30', u'300', u'3000', u'30am', u'30min', u'30mins', u'30pm', u'30s', u'30th', u

In [12]:
def display_topics(model, feature_names, n_top_words):
    for topic_index, topic in enumerate(model.components_):
        print("Topic %d:" % topic_index)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words:-1]]))

In [13]:
lda_model = LatentDirichletAllocation(n_topics=30).fit(count_review_text)
display_topics(lda_model, count_feature_names, 20)

/home/amcire/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning:

The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.



Topic 0:
got husband went waitress did came didn decided wasn friend boyfriend just said loved reviews birthday thought saw couldn
Topic 1:
cheese fries bread sandwich ordered sauce good salad meat just mac bbq taste dish like served flavor best sandwiches
Topic 2:
staff friendly super clean modern nice earl quick attentive grand job mom enjoy affordable polite lux extremely prompt especially
Topic 3:
free pricey prices stars cost tickets buy reasonable air generous offers layout shop appreciated reason high theater smells bouncers
Topic 4:
crab sushi dessert like chocolate good fresh seafood shrimp salad eat menu try dishes just taste cake items dinner
Topic 5:
surprised offer suggest bay mandalay def include pleasantly sum handle miracle nasty common pastor uni knock tsa convenience week
Topic 6:
strip vegas great like really nice good area pretty little clean hotels casino old right lot time big just
Topic 7:
room time reservation got night day arrived called went told did party mak

In [14]:
nmf_model = NMF(n_components=30).fit(tfidf_review_text)
display_topics(nmf_model, tfidf_feature_names, 20)

Topic 0:
got table came minutes said asked told took server order didn did went manager seated 10 waiter ordered experience
Topic 1:
room check bed desk bathroom stay view shower suite stayed floor night booked tower tv called clean day checked
Topic 2:
burger fries burgers shake truffle cheese onion sweet potato rings bun shack ordered bacon medium gordon shakes farm cooked
Topic 3:
great service atmosphere awesome beer experience selection location fun spot view fantastic happy loved prices recommend server definitely drinks
Topic 4:
buffet crab buffets legs selection seafood prime desserts wynn quality rib station dessert price variety dinner bellagio wicked lunch
Topic 5:
place try recommend eat awesome friends looking definitely highly packed town busy fun strip places cool menu want come
Topic 6:
wait worth line long hour waiting minutes hours 30 waited lines seated pan roast people 45 minute min prepared
Topic 7:
good really pretty nice overall price beer got little selection de

In [15]:
tagged_doc_list = []
for i in range(len(only_text)):
    text = only_text[i]
    tokens = nltk.word_tokenize(text)
    label = ordered_business_id_indices[i]
    tagged_doc = TaggedDocument(tokens, [label])
    tagged_doc_list.append(tagged_doc)

In [16]:
tagged_doc_list[0]

TaggedDocument(words=[u'I', u'love', u'sushi', u'!', u'Especially', u'with', u'cream', u'cheese', u'!', u'This', u'place', u'hardly', u'had', u'any', u'cream', u'cheese', u'sushi', u'but', u'it', u'is', u'still', u'overall', u'YUM', u'!', u':', u')', u'Service', u'is', u'excellent', u',', u'you', u'never', u'go', u'thirsty', u'as', u'they', u'keep', u'your', u'drink', u'constantly', u'full', u'!', u'Dinner', u'AYCE', u'is', u'around', u'$', u'26', u',', u'while', u'I', u"'ve", u'seen', u'lower', u'prices', u'.', u'I', u'tried', u'the', u'Philadelphia', u'roll', u',', u'caterpillar', u'and', u'Dragon', u'roll', u'.', u'philadelphia', u'roll', u'for', u'some', u'reason', u'was', u"n't", u'as', u'tasty', u'as', u'I', u'thought', u'.', u'Any', u'eel', u'in', u'the', u'rolls', u'I', u'have', u'tried', u'are', u'not', u'as', u'good', u'to', u'brag', u'about', u'honestly', u'.', u'I', u'give', u'4', u'stars', u'for', u'the', u'great', u'service', u',', u'and', u'it', u"'s", u'enough', u'to', 

In [17]:
only_text[0]

u"I love sushi! Especially with cream cheese! This place hardly had any cream cheese sushi but it is still overall YUM! :) Service is excellent, you never go thirsty as they keep your drink constantly full!\n\nDinner AYCE is around $26, while I've seen lower prices. I tried the Philadelphia roll, caterpillar and Dragon roll. philadelphia roll for some reason wasn't as tasty as I thought. Any eel in the rolls I have tried are not as good to brag about honestly. I give 4 stars for the great service, and it's enough to go back and give them another chance."

In [59]:
word2vec_dim = 500
model = Doc2Vec(size=word2vec_dim, min_count=100, iter=5)
model.build_vocab(tagged_doc_list)
%time model.train(tagged_doc_list)

CPU times: user 22min 8s, sys: 19.5 s, total: 22min 28s
Wall time: 8min 11s


171044555

In [19]:
model.save('reviews_doc2vec_model.doc2vec')


In [20]:
# load the model back
model = Doc2Vec.load('reviews_doc2vec_model.doc2vec')

In [21]:
example_index = 100

In [22]:
example_bid = rid_to_bid_dict[ordered_review_ids[example_index]]
example_rids_for_business = bid_to_rid_dict[example_bid]

In [23]:
print(len(example_rids_for_business))
example_bid

1502


u'JyxHvtj-syke7m9rbza7mA'

In [24]:

print("Review Text is:")
print(only_text[example_index])

for rid in example_rids_for_business[:5]:
    ind = ordered_review_ids.index(rid)
    print
    print("---------------")
    print("Review index %d is:" % ind)
    print(only_text[ind])

Review Text is:
Have been coming here for past year or so
great AYCE sushi place not too far from where I live

used to frequent Sushi Factory, one of the first AYCE places in town, but sadly they closed :-(

in any case, have tried a few other sushi places, have found this one to be one of the better choices in the area

usually pretty busy, never been there without having to wait...boo

their selection is pretty good, fish is fresh, haven't really had any bad meals to date
I usually try to sit at sushi bar unless with more than a couple friends, as you can usually get some good choices/tips from sushi chefs working that day

I usually stick to nigiri, so not a whole lot to explain other that the fish is usually fresh, and the ratio to rice pretty good too....make sure to check out board for specials!

love that they have salmon collar too! I do order a few of the cooked items, such as that, maybe some of the grilled specials, and oh yeah, pork belly, YUM!

I like their tuna on crispy

In [25]:
inferred = model.infer_vector(nltk.word_tokenize(only_text[100]))
inferred

array([ -7.30764747e-01,  -1.53532207e-01,   7.40111291e-01,
         3.89990866e-01,   6.31686270e-01,  -2.28908211e-01,
        -1.66462317e-01,  -1.88540407e-02,   4.25884098e-01,
        -3.05352986e-01,   6.36414662e-02,   6.20973893e-02,
         3.52771878e-01,  -3.18767995e-01,  -9.62268468e-03,
        -1.68082640e-01,   2.77332097e-01,  -1.25079766e-01,
         1.15473092e-01,  -1.55243212e-02,  -3.36315185e-01,
         1.32754713e-01,  -7.20501840e-02,   2.54140943e-01,
         2.65313089e-01,   5.39592862e-01,   2.00642403e-02,
         5.04059255e-01,  -3.17622513e-01,   2.35666737e-01,
        -9.04248506e-02,   5.58736920e-01,   1.52541876e-01,
        -7.59352967e-02,   4.30602461e-01,   7.87081942e-02,
         3.41731869e-02,  -1.72632486e-01,   6.15758859e-02,
         1.17776364e-01,   2.57349283e-01,   5.95989339e-02,
         4.09342945e-01,  -4.42171618e-02,  -8.12769830e-02,
         8.08234438e-02,   3.57433587e-01,  -2.49809623e-01,
        -1.56375337e-02,

In [26]:
sims = model.docvecs.most_similar([inferred], topn=len(model.docvecs))
sims

[(127, 0.5299594402313232),
 (141, 0.49519991874694824),
 (135, 0.4904119670391083),
 (140, 0.475095272064209),
 (32, 0.45320603251457214),
 (35, 0.4297181963920593),
 (78, 0.3670101761817932),
 (128, 0.3667944073677063),
 (3, 0.364542156457901),
 (134, 0.3492927551269531),
 (148, 0.3406960070133209),
 (30, 0.3360481858253479),
 (29, 0.3259957730770111),
 (170, 0.3046536445617676),
 (37, 0.3014053404331207),
 (147, 0.2992122769355774),
 (159, 0.2951284646987915),
 (104, 0.282723069190979),
 (8, 0.28079086542129517),
 (75, 0.28032076358795166),
 (130, 0.2793898582458496),
 (150, 0.2780572175979614),
 (99, 0.27575749158859253),
 (165, 0.2749651074409485),
 (146, 0.2749476432800293),
 (154, 0.27329137921333313),
 (155, 0.2719968855381012),
 (47, 0.2655760645866394),
 (20, 0.26232507824897766),
 (109, 0.25363287329673767),
 (133, 0.252916544675827),
 (81, 0.2525130808353424),
 (7, 0.25240999460220337),
 (106, 0.2509400248527527),
 (6, 0.2507828176021576),
 (142, 0.2505180537700653),
 (101,

In [60]:
def compute_similarities(start_ind, end_ind):
    distances = []
    c = 0.0
    sum_vectors = np.zeros(word2vec_dim)
    for i in range(start_ind, end_ind+1):
        
        text = only_text[i]
        word2vec_vector = model.infer_vector(nltk.word_tokenize(text))
        curr_business_vector = sum_vectors / max(1.0, c)
        distances.append(cosine_similarity(word2vec_vector.reshape(1,-1), curr_business_vector.reshape(1,-1))[0,0])
        sum_vectors += word2vec_vector
        c += 1
    return distances

In [28]:
bid_to_min_ind = {}
bid_to_max_ind = {}
for i, business_index in enumerate(ordered_business_id_indices):
    if business_index_to_bid[business_index] not in bid_to_min_ind:
        bid_to_min_ind[business_index_to_bid[business_index]] = i
    bid_to_max_ind[business_index_to_bid[business_index]] = i


In [29]:
bid_to_min_ind

{u'--9e1ONYQuAa-CB_Rrw7Tw': 162659,
 u'-ed0Yc9on37RoIoG2ZgxBA': 311746,
 u'0FUtlsQrJI7LhqDPxLumEw': 185134,
 u'0NmTwqYEQiKErDv4a55obg': 266425,
 u'0W4lkclzZThpx3V65bVgig': 91336,
 u'2iTsRqUsPGRH1li1WVRvKQ': 1502,
 u'2weQS-RnoOBhb1KsHKyoSQ': 62516,
 u'3BCsAgo_1i4xMuTyLKMLRQ': 99477,
 u'3GEEy7RP6e4bT4LAiWFMFQ': 125424,
 u'3N9U549Zse8UP-MwKZAjAQ': 176635,
 u'3kdSl5mo9dWC4clrQjEDGg': 48979,
 u'3l54GTr8-E3XPbIxnF_sAA': 43724,
 u'3xmfT7l3xNH5LK1dLzfvGw': 152067,
 u'4GXII-GU7S0ZyU6ElkhscQ': 123123,
 u'4JNXUYY8wbaaDmk3BPzlWw': 272498,
 u'4k3RlMAMd46DZ_JyZU0lMg': 143635,
 u'5LNZ67Yw9RD6nf4_UhXOjw': 112054,
 u'5T6kFKFycym_GkhgOiysIw': 263843,
 u'5iSmZO0SrKU6EoXK_1M8Kw': 309666,
 u'5shgJB7a-2_gdnzc0gsOtg': 102505,
 u'6Ct57qgmXwOnzfSZoUGh0Q': 172709,
 u'6Q7-wkCPc1KF75jZLOTcMw': 38324,
 u'7fxebHYUwIF6CakxSr70iQ': 31192,
 u'7sPNbCx7vGAaH7SbNPZ6oA': 56147,
 u'7sb2FYLS2sejZKxRYF9mtg': 218608,
 u'9SU7ZZhaFUJJ6m2k5HKHeg': 163970,
 u'9a3DrZvpYxVs3k_qwlCNSw': 33578,
 u'A-uZAD4zP3rRxb44WUGV5w': 242479,
 u'

In [30]:
example_bid_index = 1
example_bid = business_index_to_bid[example_bid_index]

In [31]:
similarities = compute_similarities(bid_to_min_ind[example_bid], bid_to_max_ind[example_bid])
similarities

[0.0,
 0.5292297470543551,
 0.51971354985884577,
 0.6994921496231904,
 0.70686362967581984,
 0.75879130834703878,
 0.77981162523496916,
 0.77453113751999325,
 0.68066144689204622,
 0.67606332091428389,
 0.78043190119274142,
 0.80495730446571789,
 0.78533987636029123,
 0.56585281711096269,
 0.69741377628230006,
 0.82717667618531099,
 0.70567538620372505,
 0.66095895645609615,
 0.75413027238307817,
 0.65943533697803081,
 0.76532689791968622,
 0.76248316864179744,
 0.7976829131989045,
 0.66393196069211835,
 0.63902580530970299,
 0.8056268208109314,
 0.79060055329760814,
 0.67428066779800633,
 0.73165919802783463,
 0.78936294105499971,
 0.73136736356340382,
 0.66007098717400026,
 0.66076346351409421,
 0.68823462934365731,
 0.75332323441967475,
 0.66829949499799535,
 0.71186472350988639,
 0.53007421193870607,
 0.74231478275406504,
 0.84014959717435,
 0.79096450528083384,
 0.68817776584840207,
 0.72105079082962942,
 0.73074975494813699,
 0.83316346667727426,
 0.78429642904834584,
 0.78916254

In [32]:
# Create a trace
trace = Scatter(
    x = range(len(similarities)),
    y = similarities
)

data = [trace]

py.iplot(data)

In [61]:
def compute_similarities_sliding_window(start_ind, end_ind, window_size=10):
    distances = []
    sum_vectors = np.zeros(word2vec_dim)
    vectors_in_window = Queue(maxsize=window_size)
    for i in range(start_ind, end_ind+1):        
#         print(vectors_in_window.qsize())
        text = only_text[i]
        word2vec_vector = model.infer_vector(nltk.word_tokenize(text))
        curr_business_vector = sum_vectors / max(vectors_in_window.qsize(), 1.0)
        distances.append(cosine_similarity(word2vec_vector.reshape(1,-1), curr_business_vector.reshape(1,-1))[0,0])
        if vectors_in_window.qsize() == window_size:
            popped_vector = vectors_in_window.get()
            sum_vectors -= popped_vector
        vectors_in_window.put(word2vec_vector)
        sum_vectors += word2vec_vector
        
    return distances

In [34]:
similarities_sliding_window = compute_similarities_sliding_window(bid_to_min_ind[example_bid], bid_to_max_ind[example_bid], window_size=10)
similarities_sliding_window

[0.0,
 0.48080793712883269,
 0.5281531393005976,
 0.65657272541067191,
 0.72368832649967085,
 0.71813395915426415,
 0.74823747516932437,
 0.80246392802274835,
 0.66255029896041917,
 0.69655638591312319,
 0.77760933384112385,
 0.81714271120494564,
 0.79140384968289768,
 0.57346921791124583,
 0.6476819776431243,
 0.77855292515538621,
 0.71647921593988628,
 0.63910719770415636,
 0.73779642990304173,
 0.67315507800367502,
 0.71799135838664374,
 0.74691541569828823,
 0.73914643263262214,
 0.65795179246887436,
 0.64591350495726252,
 0.80466303595465782,
 0.77003300697238297,
 0.68675532657342853,
 0.67846523451748086,
 0.76171241161040726,
 0.71578588521056963,
 0.6203476642869874,
 0.62947629304236608,
 0.67573625802722859,
 0.73519239599535224,
 0.68307801647091915,
 0.75035539854590794,
 0.545910103255004,
 0.73895756573406945,
 0.77705390861152124,
 0.78689322325987909,
 0.70893372894940465,
 0.76759798622810926,
 0.70826646341380362,
 0.84058130033229761,
 0.73051743401644065,
 0.733464

In [35]:
# Create a trace
trace = Scatter(
    x = range(len(similarities_sliding_window)),
    y = similarities_sliding_window
)

data = [trace]

py.iplot(data)

In [36]:
similarities_sliding_window_small = compute_similarities_sliding_window(bid_to_min_ind[example_bid], bid_to_max_ind[example_bid], window_size=5)
similarities_sliding_window_small

[0.0,
 0.48658705324719076,
 0.47290618704097287,
 0.68292652840193724,
 0.70229920255980205,
 0.74554192407741104,
 0.69498530853434048,
 0.74840404138997496,
 0.62073333158818567,
 0.64130565326565314,
 0.72933164379357796,
 0.76810119476029171,
 0.72792843114454253,
 0.59072218176320224,
 0.6545366445626345,
 0.78409820917687012,
 0.68696099229928975,
 0.65495053992265717,
 0.64609681142469433,
 0.65781688602837318,
 0.77183154053198555,
 0.72107514612115819,
 0.73042063627950138,
 0.64052600123731773,
 0.59363499073924908,
 0.77244765530798576,
 0.73574362059625309,
 0.69281149783963158,
 0.6835393905981253,
 0.74684822179923549,
 0.7462790050913608,
 0.62829674746033115,
 0.61090419488035064,
 0.66369234698918145,
 0.70116378596696238,
 0.64367130106715975,
 0.67316287369647265,
 0.50853308766835914,
 0.67321211604734543,
 0.75790200251500872,
 0.735636874342114,
 0.67178073709448582,
 0.68790131238261187,
 0.72507367725321026,
 0.81057050795703756,
 0.72996309896574618,
 0.683643

In [37]:
# Create a trace
trace = Scatter(
    x = range(len(similarities_sliding_window_small)),
    y = similarities_sliding_window_small
)

data = [trace]

py.iplot(data)

In [11]:
tfidf_review_text.shape

(315002, 9071)

In [12]:
word_to_index = tfidf_vectorizer.vocabulary_
index_to_word = {i:t for t,i in word_to_index.items()}
# print(word_to_index)

In [13]:
u, s, v_trans = svds(tfidf_review_text, k=100)

In [14]:
trace = Scatter(
    x = range(len(s)),
    y = s[::-1]
)

data = [trace]

py.iplot(data)

In [15]:
svd_dim = 100
reviews_compressed, _, words_compressed = svds(tfidf_review_text, k=svd_dim)
words_compressed = words_compressed.transpose()

In [16]:
print(words_compressed.shape)
print(reviews_compressed.shape)

(9071, 100)
(315002, 100)


In [17]:
from sklearn.preprocessing import normalize
words_compressed = normalize(words_compressed, axis=1)

In [18]:
def closest_words(word_in, k=10):
    if word_in not in word_to_index:
        return "Not in vocab"
    sims = words_compressed.dot(words_compressed[word_to_index[word_in],:])
    asort = np.argsort(-sims)[:k+1]
    return [(index_to_word[i], sims[i]/sims[asort[0]]) for i in asort[1:]]

In [19]:
closest_words("brunch")

[(u'mimosas', 0.96296048245179577),
 (u'bottomless', 0.95297648224633313),
 (u'mimosa', 0.94200139285947382),
 (u'champagne', 0.92622893690791674),
 (u'sunday', 0.92056258993154949),
 (u'brunches', 0.88014457829929527),
 (u'marys', 0.82561506932591522),
 (u'bellini', 0.81892137873047965),
 (u'bloody', 0.76061905142284758),
 (u'madame', 0.75592128741312137)]

In [65]:
def compute_similarities_sliding_window_svd(start_ind, end_ind, window_size=10):
    distances = []
    sum_vectors = np.zeros(svd_dim)
    vectors_in_window = Queue(maxsize=window_size)
    for i in range(start_ind, end_ind+1):        
#         print(vectors_in_window.qsize())
        text = only_text[i]
        svd_vector = reviews_compressed[i]
        curr_business_vector = sum_vectors / max(vectors_in_window.qsize(), 1.0)
        distances.append(cosine_similarity(svd_vector.reshape(1,-1), curr_business_vector.reshape(1,-1))[0,0])
        if vectors_in_window.qsize() == window_size:
            popped_vector = vectors_in_window.get()
            sum_vectors -= popped_vector
        vectors_in_window.put(svd_vector)
        sum_vectors += svd_vector
        
    return distances

In [75]:
similarities_sliding_window_svd = compute_similarities_sliding_window_svd(bid_to_min_ind[example_bid], bid_to_max_ind[example_bid], window_size=100)
similarities_sliding_window_svd

[0.0,
 -0.022562271681469959,
 0.097852569977378262,
 0.2810497404235266,
 0.066899318619547787,
 0.18149766110645793,
 0.036937287374124363,
 0.27279826726512768,
 0.089605040984308021,
 0.09038117489866157,
 0.20379702660734642,
 0.36989794906235313,
 0.35660370582225365,
 0.27170143457345253,
 0.095415144687459111,
 0.31418740174333459,
 0.33141576544283813,
 0.35269288716166297,
 0.44072744068490211,
 0.32034733323321546,
 0.10380414581274774,
 0.13623475335008631,
 0.19673265041328103,
 0.25390826567954033,
 0.48259963709042519,
 0.22564772486225393,
 0.15129194021855324,
 0.34726604026493435,
 0.19201178500425406,
 0.10724507664784294,
 0.11820921681694231,
 0.42580911950253247,
 0.34141283490253466,
 0.41524080200461916,
 0.1888537257322277,
 0.49159128903931537,
 0.38023098976349279,
 0.25660236144026305,
 0.44819506833834727,
 0.27217745647367331,
 0.2173962252686992,
 0.10292039437645163,
 -0.020967950945767588,
 0.27138908254586219,
 0.22399807718497497,
 0.41357030464135236

In [76]:
# Create a trace
trace = Scatter(
    x = range(len(similarities_sliding_window_svd)),
    y = similarities_sliding_window_svd
)

data = [trace]

py.iplot(data)